**lab 04: Naive Bayes**

Ta  có $X_i=(x_i^1,x_i^2,...,x_i^n)$ là một sample có n features, với $c$ labels ($C_1,C_2,...C_c$). Ta muốn chọn được label cho sample $X_i$ thì dựa vào:
$$C=\mathop{argmax}_{C_j, j=\overline{1,c}}P(C_j|X_i)$$
Giả sử các thành phần của biến ngẫu nhiên X độc lập với nhau, ta được: 
$$C=\mathop{argmax}_{C_j, j=\overline{1,c}}P(C_j)\prod_{i=1}^n P(X_i|C_j)$$

Tóm lại: với mỗi label $C_j$ ta tính tích $P(C_j)\prod_{i=1}^n P(X_i|C_j)$ và so sánh chúng với nhau từ đó chọn ra được label mang giá trị lớn nhất

Một số phân phối:
1. Multinomial

2. Gaussian


Bài tập
1. Dùng các features: Sex, Pclass, và Embarked để xây dựng thuật toán Naive Bayes Multinomial. (4đ)
2. Dùng các features: SibSp, Parch, và Fare để xây dựng thuật toán Naive Bayes Gaussian (Khỏi normalize). (4đ)
3. Từ bộ dữ liệu liên minh hãy xây dựng thuật toán Naive Bayes bằng cách dùng tất cả features (mix cả Multinomial và Gaussian. Lưu ý: Khỏi chia tập train, test. Dự đoán thẳng trên tập train) (2đ)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn import preprocessing
import copy

In [2]:
data = pd.read_csv('train.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Encode Sex, Embarked
data=data.drop(["Cabin","Age"],axis=1)
data=data.dropna()

encoder = preprocessing.LabelEncoder()

embarkedEncoder = copy.copy(encoder.fit(data['Embarked']))
data['Embarked'] = embarkedEncoder.transform(data['Embarked'])

sexEncoder = copy.copy(encoder.fit(data['Sex']))
data['Sex'] = sexEncoder.transform(data['Sex'])

In [4]:
# Data
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

In [5]:
def get_features(data=data_train):
    Survived=data["Survived"].values
    Pclass=data["Pclass"].values
    Sex=data["Sex"].values
    SibSp=data["SibSp"].values
    Parch=data["Parch"].values
    Embarked=data["Embarked"].values
    Fare=data["Fare"].values
    return Survived, Pclass, Sex, SibSp, Parch, Embarked,Fare

Survived_train, Pclass_train, Sex_train, SibSp_train, Parch_train,Embarked_train,Fare_train=get_features()
Survived_test, Pclass_test, Sex_test, SibSp_test, Parch_test,Embarked_test,Fare_test=get_features(data=data_test)



In [6]:
# Xác xuất tiên nghiệm.
P_Survived=np.sum((Survived_train==1))/len(Survived_train)
P_notSurvived=np.sum((Survived_train==0))/len(Survived_train)

# Sex
## Survived
P_SexisMale_Survived= np.sum((Sex_train[Survived_train==1]==0))/np.sum((Survived_train==1))
P_SexisFemale_Survived=np.sum((Sex_train[Survived_train==1]==1))/np.sum((Survived_train==1))
## Tính xác xuất hậu nghiệm trên tập (P(X|Survived) trên tập test)
P_Sex_Survived= P_SexisMale_Survived*(Sex_test==0).astype(float) +P_SexisFemale_Survived*(Sex_test==1).astype(float)

## notSurvivied
P_SexisMale_notSurvived= np.sum((Sex_train[Survived_train==0]==0))/np.sum((Survived_train==0))
P_SexisFemale_notSurvived=np.sum((Sex_train[Survived_train==0]==1))/np.sum((Survived_train==0))
## Tính xác xuất hậu nghiệm trên tập (P(X|notSurvived) trên tập test)
P_Sex_notSurvived= P_SexisMale_notSurvived*(Sex_test==0).astype(float) +P_SexisFemale_notSurvived*(Sex_test==1).astype(float)

# Pclass
## Survived
P_Pclass1_Survived= np.sum((Pclass_train[Survived_train==1]==1))/np.sum((Survived_train==1))
P_Pclass2_Survived=np.sum((Pclass_train[Survived_train==1]==2))/np.sum((Survived_train==1))
P_Pclass3_Survived=np.sum((Pclass_train[Survived_train==1]==3))/np.sum((Survived_train==1))
##Tính xác xuất hậu nghiệm trên tập (P(X|Survived) trên tập test)
P_Pclass_Survived= P_Pclass1_Survived*(Pclass_test==1).astype(float) +P_Pclass2_Survived*(Pclass_test==2).astype(float) + P_Pclass3_Survived*(Pclass_test==3).astype(float)

## notSurvivied
P_Pclass1_notSurvived= np.sum((Pclass_train[Survived_train==0]==1))/np.sum((Survived_train==0))
P_Pclass2_notSurvived=np.sum((Pclass_train[Survived_train==0]==2))/np.sum((Survived_train==0))
P_Pclass3_notSurvived=np.sum((Pclass_train[Survived_train==0]==3))/np.sum((Survived_train==0))
##Tính xác xuất hậu nghiệm trên tập (P(X|notSurvived) trên tập test)
P_Pclass_notSurvived= P_Pclass1_notSurvived*(Pclass_test==1).astype(float) +P_Pclass2_notSurvived*(Pclass_test==2).astype(float) + P_Pclass3_notSurvived*(Pclass_test==3).astype(float)

# Embarked
## Survived
P_EmbarkedS_Survived= np.sum((Embarked_train[Survived_train==1]==2))/np.sum((Survived_train==1))
P_EmbarkedC_Survived=np.sum((Embarked_train[Survived_train==1]==0))/np.sum((Survived_train==1))
P_EmbarkedQ_Survived=np.sum((Embarked_train[Survived_train==1]==1))/np.sum((Survived_train==1))
##Tính xác xuất hậu nghiệm trên tập (P(X|Survived) trên tập test)
P_Embarked_Survived= P_EmbarkedS_Survived*(Embarked_test==2).astype(float) +P_EmbarkedC_Survived*(Embarked_test==0).astype(float) + P_EmbarkedQ_Survived*(Embarked_test==1).astype(float)

## notSurvivied
P_EmbarkedS_notSurvived= np.sum((Embarked_train[Survived_train==0]==2))/np.sum((Survived_train==0))
P_EmbarkedC_notSurvived=np.sum((Embarked_train[Survived_train==0]==0))/np.sum((Survived_train==0))
P_EmbarkedQ_notSurvived=np.sum((Embarked_train[Survived_train==0]==1))/np.sum((Survived_train==0))
##Tính xác xuất hậu nghiệm trên tập (P(X|notSurvived) trên tập test)
P_Embarked_notSurvived= P_EmbarkedS_notSurvived*(Embarked_test==2).astype(float) +P_EmbarkedC_notSurvived*(Embarked_test==0).astype(float) + P_EmbarkedQ_notSurvived*(Embarked_test==1).astype(float)

#tích P(Survived|X)*P(Survived)
P_Survived_combine=P_Survived*P_Sex_Survived*P_Pclass_Survived*P_Embarked_Survived
#tích P(notSurvived|X)*P(notSurvived)
P_notSurvived_combine=P_notSurvived*P_Sex_notSurvived*P_Pclass_notSurvived*P_Embarked_notSurvived

In [7]:
np.mean((P_Survived_combine>P_notSurvived_combine).astype(float)==Survived_test)

0.7808988764044944

In [8]:
## Dùng sklearn

from sklearn.naive_bayes import CategoricalNB
# Dùng sklearn
data_train1=data_train[['Sex', 'Pclass', 'Embarked']]
data_test1=data_test[['Sex', 'Pclass', 'Embarked']]

nb = CategoricalNB().fit(data_train1, Survived_train)
nb.score(data_test1,Survived_test)

0.7808988764044944

# Bai 2

In [9]:
def normal(x, mean, std):
    return 1/(np.sqrt(2*np.pi)*std)*np.exp(-np.square(x-mean)/(2*np.square(std)))

In [10]:
# tính tham số
# Sibsp
Sibsp_mean_Survived = np.mean(SibSp_train[Survived_train==1])
Sibsp_std_Survived = np.std(SibSp_train[Survived_train==1])

Sibsp_mean_notSurvived = np.mean(SibSp_train[Survived_train==0])
Sibsp_std_notSurvived = np.std(SibSp_train[Survived_train==0])

# Parch
Parch_mean_Survived = np.mean(Parch_train[Survived_train==1])
Parch_std_Survived = np.std(Parch_train[Survived_train==1])

Parch_mean_notSurvived = np.mean(Parch_train[Survived_train==0])
Parch_std_notSurvived = np.std(Parch_train[Survived_train==0])

# Fare
Fare_mean_Survived = np.mean(Fare_train[Survived_train==1])
Fare_std_Survived = np.std(Fare_train[Survived_train==1])

Fare_mean_notSurvived = np.mean(Fare_train[Survived_train==0])
Fare_std_notSurvived = np.std(Fare_train[Survived_train==0])

# P(X|Sur) và P(X|notSur)
# Sibsp
P_SibSp_Survived = normal(SibSp_test, Sibsp_mean_Survived , Sibsp_std_Survived )
P_SibSp_notSurvived = normal(SibSp_test, Sibsp_mean_notSurvived, Sibsp_std_notSurvived)

# Parch
P_Parch_Survived = normal(Parch_test, Parch_mean_Survived , Parch_std_Survived )
P_Parch_notSurvived = normal(Parch_test, Parch_mean_notSurvived, Parch_std_notSurvived)

# Fare
P_Fare_Survived = normal(Fare_test, Fare_mean_Survived , Fare_std_Survived )
P_Fare_notSurvived = normal(Fare_test, Fare_mean_notSurvived, Fare_std_notSurvived)

# Formula
P_Survived_combine=P_Survived*P_SibSp_Survived*P_Parch_Survived*P_Fare_Survived
P_notSurvived_combine= P_notSurvived*P_SibSp_notSurvived*P_Parch_notSurvived*P_Fare_notSurvived

np.mean((P_Survived_combine>P_notSurvived_combine).astype(int)==Survived_test)

0.6910112359550562

In [11]:
## Dùng sklearn
data_train2=data_train[['SibSp', 'Parch', 'Fare']]
data_test2=data_test[['SibSp', 'Parch', 'Fare']]

nb = GaussianNB().fit(data_train2, Survived_train)
np.mean(Survived_test == nb.predict(data_test2))

0.6910112359550562

# Bai 3

Về cơ bản, Liên Minh Huyền Thoại là một game giữa hai đội Blue và Red với mục tiêu là phá hủy nhà chính đối phương. Mỗi đội có 5 người chơi, mỗi người điều khiển 1 nhân vật. Trong trò chơi, để chiến thắng người chơi có thể:

Hạ gục đối phương: thể hiện bằng chỉ số hạ gục (Kills)

Hạ gục lính kiếm tiền mua trang bị (MinionsKilled)

Cắm mắt để có tầm nhìn, thêm thông tin về đối phương (WardsPlaced)

Ăn sứ giả lấy bùa và vật phẩm (Heralds)

Ăn bùa rồng (Dragons)

Mạng đầu tiên (firstBlood)

Đội thắng (teamWins)

In [12]:
#Chuẩn bị data cho câu 3
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/dinhvietcuong1996/Lab-MachineLearningCourse/master/Lab04/lienminh.csv")
df.head()

,killsDiff,minionsKilledDiff,wardPlacedDiff,firstBlood,heralds,dragons,teamWins
0,3,-2,13,blue,none,none,red
1,0,-66,0,red,red,red,red
2,-4,-17,0,red,none,blue,red
3,-1,-34,28,red,blue,none,red
4,0,-15,58,red,none,red,red


In [13]:
encoder = preprocessing.LabelEncoder()

firstBloodEncoder = copy.copy(encoder.fit(df['firstBlood']))
df['firstBlood'] = firstBloodEncoder.transform(df['firstBlood'])

heraldsEncoder = copy.copy(encoder.fit(df['heralds']))
df['heralds'] = heraldsEncoder.transform(df['heralds'])

dragonsEncoder = copy.copy(encoder.fit(df['dragons']))
df['dragons'] = dragonsEncoder.transform(df['dragons'])

teamWinsEncoder = copy.copy(encoder.fit(df['teamWins']))
df['teamWins'] = teamWinsEncoder.transform(df['teamWins'])

In [14]:

#3
killsDiff = df['killsDiff'].values
minionsKilledDiff = df['minionsKilledDiff'].values
wardPlacedDiff = df['wardPlacedDiff'].values
firstBlood = df['firstBlood'].values
heralds = df['heralds'].values
dragons = df['dragons'].values

teamWins = df['teamWins'].values


In [15]:
# Xác xuất tiên nghiệm.
P_redWins=np.sum(teamWins==1)/len(teamWins)
P_blueWins=np.sum(teamWins==0)/len(teamWins)

# firstBlood
## redWins
P_blue_redWins= np.sum((firstBlood[teamWins==1]==0))/np.sum((teamWins==1))
P_red_redWins=np.sum((firstBlood[teamWins==1]==1))/np.sum((teamWins==1))
P_none_redWins=np.sum((firstBlood[teamWins==1]==2))/np.sum((teamWins==1))
## Tính xác xuất hậu nghiệm trên tập (P(X|Survived) trên tập test)
P_firstBlood_redWins= P_blue_redWins*(firstBlood==0).astype(float) +P_red_redWins*(firstBlood==1).astype(float)+P_none_redWins*(firstBlood==2).astype(float)

## blueWins
P_blue_blueWins= np.sum((firstBlood[teamWins==0]==0))/np.sum((teamWins==0))
P_red_blueWins=np.sum((firstBlood[teamWins==0]==1))/np.sum((teamWins==0))
P_none_blueWins=np.sum((firstBlood[teamWins==0]==2))/np.sum((teamWins==0))
## Tính xác xuất hậu nghiệm trên tập (P(X|notSurvived) trên tập test)
P_firstBlood_blueWins= P_blue_blueWins*(firstBlood==0).astype(float) +P_red_blueWins*(firstBlood==1).astype(float)+P_none_blueWins*(firstBlood==2).astype(float)

# heralds
## redWins
P_blue_redWins= np.sum((heralds[teamWins==1]==0))/np.sum((teamWins==1))
P_red_redWins=np.sum((heralds[teamWins==1]==1))/np.sum((teamWins==1))
P_none_redWins=np.sum((heralds[teamWins==1]==2))/np.sum((teamWins==1))
## Tính xác xuất hậu nghiệm trên tập (P(X|Survived) trên tập test)
P_heralds_redWins= P_blue_redWins*(heralds==0).astype(float) +P_red_redWins*(heralds==1).astype(float)+P_none_redWins*(heralds==2).astype(float)

## blueWins
P_blue_blueWins= np.sum((heralds[teamWins==0]==0))/np.sum((teamWins==0))
P_red_blueWins=np.sum((heralds[teamWins==0]==1))/np.sum((teamWins==0))
P_none_blueWins=np.sum((heralds[teamWins==1]==2))/np.sum((teamWins==0))
## Tính xác xuất hậu nghiệm trên tập (P(X|notSurvived) trên tập test)
P_heralds_blueWins= P_blue_blueWins*(heralds==0).astype(float) +P_red_blueWins*(heralds==1).astype(float)+P_none_blueWins*(heralds==2).astype(float)

# dragons
## redWins
P_blue_redWins= np.sum((dragons[teamWins==1]==0))/np.sum((teamWins==1))
P_red_redWins=np.sum((dragons[teamWins==1]==1))/np.sum((teamWins==1))
P_none_redWins=np.sum((dragons[teamWins==1]==2))/np.sum((teamWins==1))
## Tính xác xuất hậu nghiệm trên tập (P(X|Survived) trên tập test)
P_dragons_redWins= P_blue_redWins*(dragons==0).astype(float) +P_red_redWins*(dragons==1).astype(float)+P_none_redWins*(dragons==2).astype(float)

## blueWins
P_blue_blueWins= np.sum((dragons[teamWins==0]==0))/np.sum((teamWins==0))
P_red_blueWins=np.sum((dragons[teamWins==0]==1))/np.sum((teamWins==0))
P_none_blueWins=np.sum((dragons[teamWins==1]==2))/np.sum((teamWins==0))
## Tính xác xuất hậu nghiệm trên tập (P(X|notSurvived) trên tập test)
P_dragons_blueWins= P_blue_blueWins*(dragons==0).astype(float) +P_red_blueWins*(dragons==1).astype(float)+P_none_blueWins*(dragons==2).astype(float)


In [16]:
#tích P(redWins|X)*P(redWins)
P_redWins_combine=P_redWins*P_firstBlood_redWins*P_heralds_redWins*P_dragons_redWins
#tích P(notSurvived|X)*P(notSurvived)
P_blueWins_combine=P_blueWins*P_firstBlood_blueWins*P_heralds_blueWins*P_dragons_blueWins

In [17]:
np.mean((P_redWins_combine>P_blueWins_combine).astype(float)==teamWins)

0.612005263690657

In [18]:
from sklearn.naive_bayes import CategoricalNB
# Dùng sklearn
data=df[['firstBlood', 'heralds', 'dragons']]


nb = CategoricalNB(alpha=1.0e-10).fit(data, teamWins)
nb.score(data, teamWins)

0.6298208320680231

In [19]:
def normal(x, mean, std):
    return 1/(np.sqrt(2*np.pi)*std)*np.exp(-np.square(x-mean)/(2*np.square(std)))

In [20]:
#Tính tham so
#killsDiff
killsDiff_mean_redWins = np.mean(killsDiff[teamWins==1])
killsDiff_std_redWins = np.std(killsDiff[teamWins==1])

killsDiff_mean_blueWins = np.mean(killsDiff[teamWins==0])
killsDiff_std_blueWins = np.std(killsDiff[teamWins==0])
#minionsKilledDiff
minionsKilledDiff_mean_redWins = np.mean(minionsKilledDiff[teamWins==1])
minionsKilledDiff_std_redWins = np.std(minionsKilledDiff[teamWins==1])

minionsKilledDiff_mean_blueWins = np.mean(minionsKilledDiff[teamWins==0])
minionsKilledDiff_std_blueWins = np.std(minionsKilledDiff[teamWins==0])
#wardPlacedDiff
wardPlacedDiff_mean_redWins = np.mean(wardPlacedDiff[teamWins==1])
wardPlacedDiff_std_redWins = np.std(wardPlacedDiff[teamWins==1])

wardPlacedDiff_mean_blueWins = np.mean(wardPlacedDiff[teamWins==0])
wardPlacedDiff_std_blueWins = np.std(wardPlacedDiff[teamWins==0])

In [21]:
#Tinh normalDistribution
#killsDiff
P_killsDiff_redWins = normal(killsDiff, killsDiff_mean_redWins , killsDiff_std_redWins)
P_killsDiff_blueWins = normal(killsDiff, killsDiff_mean_blueWins, killsDiff_std_blueWins)

#minionsKilledDiff
P_minionsKilledDiff_redWins = normal(minionsKilledDiff, minionsKilledDiff_mean_redWins , minionsKilledDiff_std_redWins)
P_minionsKilledDiff_blueWins = normal(minionsKilledDiff, minionsKilledDiff_mean_blueWins,minionsKilledDiff_std_blueWins)

#wardPlacedDiff
P_wardPlacedDiff_redWins = normal(wardPlacedDiff, wardPlacedDiff_mean_redWins , wardPlacedDiff_std_redWins)
P_wardPlacedDiff_blueWins = normal(wardPlacedDiff, wardPlacedDiff_mean_blueWins, wardPlacedDiff_std_blueWins)


In [22]:
# Formula
P_redWins_combine=P_redWins * P_killsDiff_redWins * P_minionsKilledDiff_redWins * P_wardPlacedDiff_redWins
P_blueWins_combine= P_blueWins * P_killsDiff_blueWins * P_minionsKilledDiff_blueWins * P_wardPlacedDiff_blueWins

np.mean((P_redWins_combine>P_blueWins_combine).astype(float)==teamWins)

0.7082700678206296

In [23]:
from sklearn.naive_bayes import GaussianNB
# Dùng sklearn
data=df[['killsDiff', 'minionsKilledDiff', 'wardPlacedDiff']]


nb = GaussianNB().fit(data, teamWins)
np.mean(teamWins == nb.predict(data))

0.7082700678206296